In [53]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '..')

import casadi as ca
from pyecca.lie.se2 import SE2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
p1 = ca.SX.sym('p1', 3)
p2 = ca.SX.sym('p2', 3)

In [56]:
SE2.product(SE2.inv(p1), p1)

SX(@1=(-p1_2), @2=cos(@1), @3=sin(@1), @4=cos(p1_2), @5=sin(p1_2), [(((@2*p1_0)-(@3*p1_1))-((@4*p1_0)+(@5*p1_1))), (((@3*p1_0)+(@2*p1_1))+((@5*p1_0)-(@4*p1_1))), 0])

In [60]:
u = SE2.log(SE2.product(SE2.inv(p1), p2))
u

SX(@1=(p2_2-p1_2), @2=1e-07, @3=(fabs(@1)<@2), @4=1, @5=((@3?((@4+(-0.166667*sq(@1)))+(0.00833333*sq(sq(@1)))):0)+((!@3)?(sin(@1)/@1):0)), @6=(fabs(@1)<@2), @7=((@6?(((0.5*@1)+(-0.0416667*(@1*sq(@1))))+(0.00138889*(@1*sq(sq(@1))))):0)+((!@6)?((@4-cos(@1))/@1):0)), @8=(sq(@5)+sq(@7)), @9=(-p1_2), @10=cos(@9), @11=sin(@9), @12=cos(p1_2), @13=sin(p1_2), @14=(((@10*p2_0)-(@11*p2_1))-((@12*p1_0)+(@13*p1_1))), @15=(((@11*p2_0)+(@10*p2_1))+((@13*p1_0)-(@12*p1_1))), [(((@5/@8)*@14)+((@7/@8)*@15)), (((@5/@8)*@15)-((@7/@8)*@14)), @1])